### Saveing the data in a model:

1. Create a model and give it a name.
2. Create the needed fields, and set their arguments to what you want.
3. Modify the admin.py file so you can manage the model in your administrative section.

Your models.py file should look like the following block:

In [ ]:
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator

class Review(models.Model):
    user_name = models.CharField(max_length=100)
    review_text = models.TextField()
    rating = models.IntegerField(validators=[MinValueValidator(0), MaxValueValidator(10)])

    def __str__(self):
        return f'Review by {self.user_name} - Rating: {self.rating}'
    
    class Meta:
        verbose_name_plural = 'All Reviews'

Your admin.py file should look like the following block:

from django.contrib import admin
from .models import Review
# Register your models here.


class ReviewAdmin(admin.ModelAdmin):
    list_display = ('user_name', 'rating')
    list_filter = ['rating']

admin.site.register(Review, ReviewAdmin)

Save everything, makemigrations, and migrate.

Now, you can go to the views.py and use the data which is provided by the user and save them in the database.<br>

Please note that you need to do it after validationg the form<br>

Your views.py file should look like the following block:

In [ ]:
from django.shortcuts import render
from django.http import HttpResponseRedirect
from .forms import ReviewForm
from .models import Review

def index(request):
    if request.method =='POST':
        form = ReviewForm(request.POST)
        if form.is_valid():
            review = Review(user_name=form.cleaned_data['user_name'],
                            review_text=form.cleaned_data['review_text'],
                            rating=form.cleaned_data['rating'])
            review.save()
            return HttpResponseRedirect('/results')
    else:
        form = ReviewForm()
    return render(request, 'reviews/index.html',{
        'form':form
    })


def res(request):
    return render(request, 'reviews/res.html')

Now save everything, reload the page, and submit the form. Then check the admin page to see if evrything works correctly.<br>

There is one thing I want to talk about here!

You may have concluded that in our future projects, we are going to create forms and models as pairs for atleast 90% of cases! This is so common that django has built a feature where it generated forms based on models! You can use this feature in you advantage to write less code.<br>

How does it work?<br>
Let me explain it step by step:<br>
1. Navigate to your forms.py file
2. Delete or comment your ReviewForm class
3. Create a new ReviewForm class and extend it from forms.Model.Form
4. Now, you need to connect it to a model.
    - Create a Meta class inside your ReviewForm class
    - Create an attribute inside it called model and set it to the Review model (You need to mport your model)
    - You might have a field that should not be included inside the form, hence you need specify all the fields that you wish to be included in the form,
        - Create an attribute called fields and set it to a list of all the fields name that you want to be included (name of the fields should be entered as strings).
        - If you want to add all the fields, you dont need to name them all, instead you can use a special identifier named '__all__' which is a string for the fields attribute.
        - If you want to include all the fields, except for one or two, you could use exclude insted of fields and set to to list of all fields that you want to exclude.

Your forms.py file should look like the following block:

In [ ]:
from django import forms
from .models import Review

class ReviewForm(forms.ModelForm):
    class Meta:
        model = Review
        # fields = '__all__'
        fields = ['user_name', 'review_text', 'rating']
        # exclude = ['field1', 'field2', ...]

Save everything and reload the page to see the changes.<br>

You might have noticed that you have lost your labels! it is due to the fact that django is now creating your fields and not you.<br>

Don't worry, in the next notebook you will learn how to configure an auto generated form.